<a href="https://colab.research.google.com/github/HyoJaDan/VGGNet/blob/main/%08VGGNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
batch_size = 100
learning_rate = 0.0002
num_epoch = 100

# Transform 정의
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10 TRAIN 데이터 정의
cifar10_train = datasets.CIFAR10(root="../Data/", train=True, transform=transform, target_transform=None, download=True)

# CIFAR10 TEST 데이터 정의
cifar10_test = datasets.CIFAR10(root="../Data/", train=False, transform=transform, target_transform=None, download=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


train_loader = torch.utils.data.DataLoader(cifar10_train,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(cifar10_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)
#받아온 데이터를 학습하기 위해 나누어줍니다.
#batch_size선언, shuffle : 데이터를 무작위로 섞을때
#num_workers : 데이터를 묶을때 사용하는 프로세스 갯수
#drop_last : 묶고 남은 자투리 데이터들은 버릴지 말지


def conv_2_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model
def conv_3_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model
class VGG(nn.Module):
    def __init__(self, base_dim, num_classes=10):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3,base_dim), #64
            conv_2_block(base_dim,2*base_dim), #128
            conv_3_block(2*base_dim,4*base_dim), #256
            conv_3_block(4*base_dim,8*base_dim), #512
            conv_3_block(8*base_dim,8*base_dim), #512
        )
        self.fc_layer = nn.Sequential(
            # CIFAR10은 크기가 32x32이므로
            nn.Linear(8*base_dim*1*1, 4096),
            # IMAGENET이면 224x224이므로
            # nn.Linear(8*base_dim*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(1000, num_classes),
        )

    def forward(self, x):
        x = self.feature(x)

        x = x.view(x.size(0), -1)

        x = self.fc_layer(x)

        return x


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  #이부분은 굳이 안해주셔도 됩니다. GPU를 사용할 수 없는경우 CPU를 쓰겠다는 것으로, 이부분을 주석처리하고
  # model = CNN()로만 해주셔도 됩니다.
model = VGG(base_dim=64).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
#Cross Entropy Loss function, Adam optimizer

loss_arr = []
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        #mnist 학습용 data를 불러옵니다.(28x28)
        y_ = label.to(device)
        #각각의 data들이 0~9중 어떤숫자인지도 불러옵니다.
        optimizer.zero_grad()
        #optimizer 초기화
        output = model.forward(x)

        #학습용 데이터로 CNN 실시
        loss = loss_func(output,y_)
        #학습해서 추정해낸 값과, 실제 라벨된 값 비교
        loss.backward()
        #오차만큼 다시 Back Propagation 시행
        optimizer.step()
        #Back Propagation시 ADAM optimizer 매 Step마다 시행
        if j % 1000 == 0 :
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())
correct = 0
total = 0

with torch.no_grad():
    for image,label in test_loader :
        x = image.to(device)
        y_ = label.to(device)
        output = model.forward(x)
        _,output_index = torch.max(output,1)
        total += label.size(0)
        correct += (output_index == y_).sum().float()
    print("Accuracy of Test Data : {}".format(100*correct/total))

Files already downloaded and verified
Files already downloaded and verified
tensor(2.3027, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9017, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4814, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3852, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2654, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8127, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8560, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7536, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5218, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4254, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2794, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2843, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1841, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1720, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2329, device='c